In [ ]:
# Predict the centered cell
from utils import *
os.environ['CUDA_VISIBLE_DEVICES'] = "1" # Set the gpu card number to use. Use this line if your machine has multiple gpu cards.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' for FATAL logs only

In [ ]:
# Parameters
nr = 5 # grid size of nr*nr
Capacity_forecasting = 1 # 1: Capacity forecasting, 0: traffic forecasting
lookback = 3 # history length
alpha = 0.1 # calibration parameter for capacity forecasting

if Capacity_forecasting == 1:
    damage = 'capacity_forecasting'
    loss_func = cost_func_more_args(alpha)
else:
    damage = 'mae'
    loss_func = mae

cell = 5355 # select the center cell to perform the attack in the 5*5 grid around it

In [ ]:
# DeepCog architecture from: https://github.com/wnlUc3m/deepcog
sample_shape = (nr, nr)
batch_size = 128
num_cluster = 1
no_epochs = 150
validation_split = 0.2
verbosity = 1
#steps_per_epoch = 40
neurons = 32
ker_sz = 3

def make_nn_model(load_input_shape, lookback, num_cluster):
    '''Build DeepCog architecture'''
    inputs = tf.keras.layers.Input(shape=(
        lookback,  load_input_shape[0],load_input_shape[1],  1))
    x = tf.keras.layers.Conv3D(neurons, kernel_size = (ker_sz, ker_sz, ker_sz), activation = 'relu', padding = 'same', name = 'block1_conv1')(inputs)
    x = tf.keras.layers.Conv3D(neurons, kernel_size = (ker_sz * 2, ker_sz * 2, ker_sz * 2), activation = 'relu', padding = 'same')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Conv3D(neurons / 2, kernel_size=(ker_sz * 2,ker_sz * 2, ker_sz * 2), activation = 'relu', padding = 'same')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(neurons * 2, activation='relu')(x)
    x = tf.keras.layers.Dense(neurons, activation='relu')(x)
    output = tf.keras.layers.Dense(num_cluster)(x)
    model = Model(inputs = inputs, outputs = output)
    if Capacity_forecasting == False:
        model.compile(optimizer = Adam(0.0005), loss = mae)
    else:
        model.compile(optimizer = Adam(0.0005), loss = cost_func_more_args(alpha))
    return model


In [ ]:
# load data and models

nn = make_nn_model(sample_shape, lookback, num_cluster)
if Capacity_forecasting == True:
    nn = tf.keras.models.load_model('../Trained_models/'+str(damage)+'/Models/mymodel_%d.h5' %cell,
                                    custom_objects = {'loss': cost_func_more_args}, compile = False)
else:
    nn = tf.keras.models.load_model('../Trained_models/'+str(damage)+'/Models/mymodel_%d.h5' %cell,
                                    custom_objects = {'loss': mae}, compile = False)
    
test = np.load('../Trained_models/'+str(damage)+'/Data/test_'+ str(cell)+'.npy')

ds = []
ds2 = []
tmp = []
tmp2 = []

ds= tf.keras.preprocessing.timeseries_dataset_from_array(test, targets = None, sequence_length = lookback,
                                                            sequence_stride = 1, shuffle = False)
for batch in ds:
    tmp.append(batch.numpy())
test_X=np.vstack(tmp)

ds2= tf.keras.preprocessing.timeseries_dataset_from_array(test[:,2,2], targets = None, sequence_length = 1,
                                                            sequence_stride = 1, shuffle = False, start_index = lookback)
for batch2 in ds2:
    tmp2.append(batch2.numpy())
test_Y = np.vstack(tmp2)
test_X = test_X[0:test_Y.shape[0]]

In [ ]:
# attack parameters
x1 = [0, 25, 50, 100, 300] # attack start time
duration = [100, 400, 800, 1000] # attack duration  
epsilon = [0.01, 0.06, 0.09, 0.2] # fgsm/bim epsilon
LEN = len(x1)

In [ ]:
for eps in epsilon:        
    for d in (duration):
        max_traffic = []
        real_traffic = []      
        print('duration:', d)
        print('epsilon:', eps)           
        
        FGSM_SLA_NUM = 0
        BIM_SLA_NUM = 0
        prediction_SLA_NUM = 0
        LRP_SLA_NUM = 0  
        LRP_least_SLA_NUM = 0

        FGSM_OVERPROV_COST = 0
        BIM_OVERPROV_COST = 0
        prediction_OVERPROV_COST = 0
        LRP_OVERPROV_COST = 0  
        LRP_least_OVERPROV_COST = 0
   
        for start in (x1):                         
            x2 = start + d   
            test_XX = test_X[start:x2,:,:,:]
            test_YY = test_Y[start:x2,:]       
            # load most/least relevant cell ids
            cell_ids = np.load('./cell_ids/'+str(cell)+'_'+str(damage)+'_most_relevant.npy')
            cell_ids2 = np.load('./cell_ids/'+str(cell)+'_'+str(damage)+'_least_relevant.npy')

            test_YY = np.reshape(test_YY, (test_YY.shape[0],1))
            
            prediction_conv = nn.predict(test_XX) # predicted 
            ###################################################################################
            # normal FGSM attack
            adv_X, _ = fgsm(X = test_XX, Y = test_YY, model = nn , loss_fn = loss_func,
                            epsilon = eps, targeted = True)
            bim_X, _ = bim(X = test_XX, Y = test_YY, model = nn , loss_fn = loss_func,
                            epsilon = eps, alpha2 = 0.001, I = 200, targeted = True)
            FGSM = nn.predict(adv_X)
            BIM = nn.predict(bim_X)
            test_X2 = test_XX.copy()
            test_X3 = test_XX.copy()
            for element in range(0, len(adv_X)):
                traffic_peak = max(test_YY[element])                
                for h in range(2,3):
                    added_value = 0
                    added_value2 = 0
                    injected_element = np.sum(adv_X[element,h,:,:] - test_XX[element,h,:,:])
                    idx = int(cell_ids[element])
                    row = math.floor(idx / 5)
                    col = idx % 5
                    added_value = injected_element + test_XX[element,h,row,col]
                    test_X2[element,h,row,col] = added_value

                LRP = nn.predict(test_X2)
                ###############################################################################                                             
                for h in range(2,3):
                    injected_element = np.sum(adv_X[element,h,:,:] - test_XX[element,h,:,:])
                    #LRP least: Approach 1 
                    idx2 = int(cell_ids2[element])
                    row2 = math.floor(idx2 / 5)
                    col2 = idx2 % 5
                    added_value2 = injected_element + test_XX[element,h,row2,col2]
                    test_X3[element,h,row2,col2] = added_value2

                LRP_least = nn.predict(test_X3)
                ###############################################################################
                FGSM_sla_num = sla_num(FGSM[element], test_YY[element])
                BIM_sla_num = sla_num(BIM[element], test_YY[element])
                prediction_sla_num = sla_num(prediction_conv[element], test_YY[element])
                LRP_sla_num = sla_num(LRP[element], test_YY[element])     
                LRP_least_sla_num = sla_num(LRP_least[element], test_YY[element])
                            
                FGSM_overprov_cost = overprov_cost(FGSM[element], test_YY[element], traffic_peak, alpha)
                BIM_overprov_cost = overprov_cost(BIM[element], test_YY[element], traffic_peak, alpha)
                prediction_overprov_cost = overprov_cost(prediction_conv[element], test_YY[element], traffic_peak, alpha)
                LRP_overprov_cost = overprov_cost(LRP[element], test_YY[element], traffic_peak, alpha)     
                LRP_least_overprov_cost = overprov_cost(LRP_least[element], test_YY[element], traffic_peak, alpha)

                FGSM_SLA_NUM += FGSM_sla_num
                BIM_SLA_NUM += BIM_sla_num
                prediction_SLA_NUM += prediction_sla_num
                LRP_SLA_NUM += LRP_sla_num   
                LRP_least_SLA_NUM += LRP_least_sla_num 

                FGSM_OVERPROV_COST += FGSM_overprov_cost
                BIM_OVERPROV_COST += BIM_overprov_cost
                prediction_OVERPROV_COST += prediction_overprov_cost
                LRP_OVERPROV_COST += LRP_overprov_cost 
                LRP_least_OVERPROV_COST += LRP_least_overprov_cost

        print('')
        print('FGSM_SLA_num', (FGSM_SLA_NUM/LEN))
        print('BIM_SLA_num', (BIM_SLA_NUM/LEN))
        print('LRP_SLA_num', (LRP_SLA_NUM/LEN))
        print('LRP_least_SLA_num', (LRP_least_SLA_NUM/LEN))
        print('no_attack_SLA_num:', (prediction_SLA_NUM/LEN))
        print('')       

        print('FGSM_OVERPROV_cost', float("{:.2f}".format((FGSM_OVERPROV_COST/LEN))))
        print('BIM_OVERPROV_cost', float("{:.2f}".format((BIM_OVERPROV_COST/LEN))))
        print('LRP_OVERPROV_cost', float("{:.2f}".format((LRP_OVERPROV_COST/LEN))))
        print('LRP_least_OVERPROV_cost', float("{:.2f}".format((LRP_least_OVERPROV_COST/LEN))))
        print('no_attack_OVERPROV_cost:', float("{:.2f}".format((prediction_OVERPROV_COST/LEN))))
        print('')
        print('-----------------------------------------------------------------------------')
      